In [ ]:
cap_dict = {}
#input
# CLASS,L1,L2,L3,L4,L5,L6
# 1, 2300, 4300, 6300, 8300, 10300, 12300
#disct: (class,lanes)=capacity eg (39,6)=9420
with open('TRTS4S_Capacity.csv', 'r') as f:
    lines = f.readlines()
    for line in lines[1:]:  # Skip the header line
        parts = line.strip().split(',')
        class_num = int(parts[0])
        for i in range(1, len(parts)):
            lanes = i
            capacity = int(parts[i])
            cap_dict[(class_num, lanes)] = capacity

In [ ]:
#調撥車道處理
#no need to look up capacity
#open Y110_modified_v7.shp
gdf = gpd.read_file('Y110_modified_v7.shp')

# 承德路7:00-9:00
# [8299,6666]:(5,3) means (A,B)=(8299,6666), LANES_110=5; (A,B)=(6666,8299), LANES_110=3
# [6666,6628,8293,15665,9978,8289]:(4,2)
# [6631,6660,6642,8141,14039,8143,9883,6387]:(5,3)
# 中山北路7:00-9:00
# [6291,6696,6695,6629,8291]:(3,1)
# 民族東路7:00-9:00
# [8345,6682,9154,15473,6375,6683]:(3,1)
# [6683,6373,6409,6398]:(4,2)
# 復興北路 五常街至民權東路 07:00~09:00
# [6753,8090]:(4,2)

reversible_links_dict = {
    (8299, 6666): (5, 3),
    (6666, 6628, 8293, 15665, 9978, 8289): (4, 2),
    (6631, 6660, 6642, 8141, 14039, 8143, 9883, 6387): (5, 3),
    (6291, 6696, 6695, 6629, 8291): (3, 1),
    (8345, 6682, 9154, 15473, 6375, 6683): (3, 1),
    (6683, 6373, 6409, 6398): (4, 2),
    (6753, 8090): (4, 2)
}

# 將可調撥車道轉換為字典形式
reversible_links = {}
for key, value in reversible_links_dict.items():
    if isinstance(key, tuple):
        for i in range(len(key) - 1):
            reversible_links[(key[i], key[i + 1])] = value[0]  # 使用第一個值作為車道數
            reversible_links[(key[i + 1], key[i])] = value[1]  # 使用第二個值作為車道數
    else:
        reversible_links[key] = value[0]  # 單一連結的情況
        reversible_links[(key[1], key[0])] = value[1]  # 反向連結

print("Reversible links:", reversible_links)

# 遍歷每一行數據
for index, row in gdf.iterrows():
    # 獲取 A 和 B 的值
    A = row['A']
    B = row['B']
    
    # 獲取車道數
    lanes = row['LANES_110']
    
    # 獲取 CLASS_110
    class_num = row['CLASS_110']
    
    # 檢查是否為可調撥車道
    if (A, B) in reversible_links:
        lanes = reversible_links[(A, B)]
        capacity = cap_dict.get((class_num, lanes), 0)
    elif (B, A) in reversible_links:
        lanes = reversible_links[(B, A)]
        capacity = cap_dict.get((class_num, lanes), 0)
    else:
        capacity = cap_dict.get((class_num, lanes), 0)
    
    # 更新行數據(LANES_110, capacity)
    gdf.at[index, 'LANES_110'] = lanes
    gdf.at[index, 'capacity'] = capacity

# 保存修改後的 GeoDataFrame
gdf.to_file('Y110_modified_v7_updated.shp', driver='ESRI Shapefile')

In [ ]:
# Network

input_file = 'Y110_TRTS4S_Net_v7.csv'
output_file = 'network.dat'

df = pd.read_csv(input_file)
# if there is '', then replace with 0
df_out = pd.DataFrame(columns=['origin','dest','capacity','length','fft','alpha','beta','speedLimit'])
df.fillna(0, inplace=True)
rows = []

for index, row in df.iterrows():
    if row["LEVEL_110"] in [1,2,3,4,5,6,7,11]:# and row["CLASS_110"] #not in [i for i in range(84, 99)]+[61]:
        # if DIR=1, then A is origin, B is dest
        # if DIR=-1, then A is dest, B is origin
        # if DIR=2, then the road is bidirectional, so we need to add two rows
        # if row["DIR"] == 1 or row["DIR"] == 2:
            # print(row["ID"], row["DIR"], row["A"], row["B"])
        origin = row["A"]
        dest = row["B"]
        # elif row["DIR"] == -1:
            # print(row["ID"], row["DIR"], row["A"], row["B"])
            # origin = row["B"]
            # dest = row["A"]
        capacity = cap_dict.get((row["CLASS_110"], row["LANES_110"]), 0)
        if row["CLASS_110"] == 99:
            capacity = 9999
        #4359,8218,8251,8252,3677,8260,10923,8270,8271,10928,3646,4018 ID, half capacity
        if row["ID"] in [4359, 8218, 8251, 8252, 3677, 8260, 10923, 8270, 8271, 10928, 3646, 4018]:
            capacity = capacity/2
        #2343 ID, 0.85 capacity
        if row["ID"] == 2343:
            capacity = capacity*0.85
        length = float(row["LENGTH"])  or 1000
        try:
            fft = length/float(row["S0"])*60
        except:
            # if ID is not 40438, then give S0=32
            # print(row["ID"], row["S0"])
            if row["ID"] != 40438:
                fft = length/32*60
            else:
                fft = 1000
        alpha = row["ALPHA"] or 0
        beta = row["BETA"] or 0
        speedLimit = 0
        if capacity == 0:
            continue
        rows.append({'origin':origin,'dest':dest,'capacity':capacity,'length':length,'fft':fft,'alpha':alpha,'beta':beta,'speedLimit':speedLimit})
        # if row["DIR"] == 2:
            # add the reverse direction
            # rows.append({'origin':dest,'dest':origin,'capacity':capacity,'length':length,'fft':fft,'alpha':alpha,'beta':beta,'speedLimit':speedLimit})

# write in df_supplement from network_supplement.dat
df_supplement = pd.read_csv('network_supplement.dat', sep='\t', header=None, names=['origin','dest','capacity','length','fft','alpha','beta','speedLimit'])
# set columns=['origin','dest','capacity','length','fft','alpha','beta','speedLimit']

# tail.node	head.node	capacity..veh.h.		fftt.min.			speed.limit..mph.

df_out = pd.concat([df_out, pd.DataFrame(rows)], ignore_index=True)
df_out = pd.concat([df_out, df_supplement], ignore_index=True)

df_out.to_csv(output_file, index=False, sep='\t')